In [1]:
import pandas as pd
import numpy as np
import datetime as datetime

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import json
from pathlib import Path
from DPM.task_free_continual_learning_arf.main_provide_data_model import main
from experiments.config import ConfigParser
from experiments.config import Recorder
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import time
import pandas as pd
import numpy as np
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
import edbn.Methods.method as method
from edbn.Methods.SDL import sdl

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## old way

In [3]:
df_orig = pd.read_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/BPI_Challenge_2020/PermitLog.csv')

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8,12,15,20,21,22,23,25,26,30,32,33,35,37,38,39,41,42,43,44,45,46,47,48,49,51,52,53,54,55,58,61,62,63,64,65,66,67,68,73,74,77,80,81,82,83,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,167,168,169,170,171,172) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [73]:
df_orig.columns

Index(['id', 'org:resource', 'event', 'completeTime', 'role',
       'case:OrganizationalEntity', 'case:ProjectNumber', 'case:TaskNumber',
       'case:dec_id_0', 'case:ActivityNumber',
       ...
       'case:Cost Type_10', 'case:Cost Type_11', 'case:Cost Type_12',
       'case:Task_5', 'case:Task_4', 'case:Task_9', 'case:Task_8',
       'case:Task_7', 'case:Task_6', 'year'],
      dtype='object', length=174)

In [74]:
df=df_orig

In [6]:
df['completeTime']=pd.to_datetime(df['completeTime'])

In [7]:
datetime_obj = pd.to_datetime(df['completeTime'], "%Y-%m-%d %H:%M:%S+00:00")

# Extract the year
df['year'] = datetime_obj.dt.year

In [15]:
df.loc[df['year'] == 2018].index

Int64Index([  112,   113,   114,   115,   116,   148,   149,   600,   601,
              639,
            ...
            86552, 86553, 86558, 86559, 86563, 86564, 86569, 86570, 86575,
            86576],
           dtype='int64', length=71179)

## processed logfile

In [7]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/BPI_Challenge_2020/PrepaidTravelCost_new.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])

Num events is 18246


In [16]:
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
print('data prepared done')

start_time = time.time()
print("Test Context Data")
#print(d.test_orig.contextdata)
# basic_model = m.train(d.train,{"epochs": 0, "early_stop": 10})

connect_symbol="-"
if '/' in d.logfile.get_data()['completeTime'][0]:
    connect_symbol='/'
formats = [f"%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S%z", f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S',f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S.%f']

for timeformat in formats:
    try:
        pd.to_datetime(d.logfile.get_data()['completeTime'], format=timeformat,exact=True)
        print(d.logfile.get_data()['completeTime'])
        print("The time format is:", timeformat)
        break
    except ValueError:
        continue
print('timeformat',timeformat)
#d.add_data_to_test_orig([20,50],timeformat)

PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 0
Test: 18246
data prepared done
Test Context Data
0        2017-01-09 13:48:43+00:00
1        2017-01-09 13:48:43+00:00
2        2017-01-09 13:48:55+00:00
3        2017-01-09 13:48:55+00:00
4        2017-01-10 10:19:16+00:00
                   ...            
18241    2019-01-17 16:31:44+00:00
18242    2019-01-25 09:15:02+00:00
18243    2019-01-28 16:31:29+00:00
18244    2019-02-04 16:31:26+00:00
18245    2019-02-21 10:11:10+00:00
Name: completeTime, Length: 18246, dtype: object
The time format is: %Y-%m-%d %H:%M:%S%z
timeformat %Y-%m-%d %H:%M:%S%z


In [13]:
# d.add_data_to_test_orig([20,50],timeformat)

In [17]:
df = d.logfile.get_data()
df.head()

,index,event_Prev9,role_Prev9,completeTime_Prev9,case_Prev9,event_Prev8,role_Prev8,completeTime_Prev8,case_Prev8,event_Prev7,...,completeTime_Prev1,case_Prev1,event_Prev0,role_Prev0,completeTime_Prev0,case_Prev0,event,role,completeTime,case
0,15436,0,0,0,1756,0,0,0,1756,0,...,0,1756,0,0,0,1756,14,4,2017-01-09 13:48:43+00:00,1756
1,15442,0,0,0,1757,0,0,0,1757,0,...,0,1757,0,0,0,1757,14,4,2017-01-09 13:48:43+00:00,1757
2,15443,0,0,0,1757,0,0,0,1757,0,...,0,1757,14,4,2017-01-09 13:48:43+00:00,1757,7,7,2017-01-09 13:48:55+00:00,1757
3,15437,0,0,0,1756,0,0,0,1756,0,...,0,1756,14,4,2017-01-09 13:48:43+00:00,1756,7,7,2017-01-09 13:48:55+00:00,1756
4,15989,0,0,0,1823,0,0,0,1823,0,...,0,1823,0,0,0,1823,14,4,2017-01-10 10:19:16+00:00,1823


In [18]:
# Convert the series to a datetime type
data = pd.to_datetime(df['completeTime'])

# Remove timezone information
data = data.dt.tz_localize(None)

# Filter the series based on the desired date range
start_date = pd.to_datetime("2017-12-24").tz_localize(None)
end_date = pd.to_datetime("2018-02-14").tz_localize(None)
filtered_data = data[(data >= start_date) & (data <= end_date)]

# Find the index of the last occurrence in the filtered data
first_occurrence_index = filtered_data.idxmin()
last_occurrence_index = filtered_data.idxmax()

print("first_occurrence_index", first_occurrence_index,"Last occurrence index:", last_occurrence_index)

first_occurrence_index 2367 Last occurrence index: 4473


In [18]:
filtered_data

9878    2017-12-26 04:15:05
9879    2017-12-28 04:15:00
9880    2017-12-28 04:15:00
9881    2018-01-02 08:18:55
9882    2018-01-02 08:25:21
                ...        
14497   2018-02-13 19:46:12
14498   2018-02-13 19:46:19
14499   2018-02-13 21:47:52
14500   2018-02-13 22:43:51
14501   2018-02-13 22:50:48
Name: completeTime, Length: 4624, dtype: datetime64[ns]

In [4]:
9878/56437

0.1750270212803657

In [103]:
print(first_values_new['index'].min(),first_values_old['index'].max())

15923 15886


## old way

In [43]:
df.reset_index().groupby('case')[['index','year']]

In [34]:
# Group the DataFrame by 'case' and sort the rows within each group by 'completeTime'
#df_sorted = df.groupby('case').apply(lambda x: x.sort_values('completeTime')).reset_index(drop=True)

# Find the first value of 'completeTime' in each group
first_values = df.reset_index().groupby('case')[['index','year']].first()

In [40]:
first_values_new = first_values[first_values['year']==2018]

In [42]:
first_values_new['index'].min()

15923

In [35]:
first_values.groupby('year').min()

,index
year,
2016,0
2017,117
2018,15923


In [22]:
last_values = df.reset_index().groupby('case')[['index','year']].last()

In [23]:
last_values.groupby('year').max()

,index
year,
2017,15165
2018,86271
2019,86580
2020,44922
2021,52193


In [26]:
# Assuming your dataframe is named "df"
last_event_index = df[df['year'] == 2017].index[-1]

In [27]:
last_event_index

15886

In [37]:
first_values_old = first_values[first_values['year'] == 2017]

In [39]:
first_values_old['index'].max()

15886

In [72]:
# Filter the dataframe to cases starting in 2017
filtered_df = df[df['year'] == 2017]


last_values = filtered_df.reset_index().groupby('case')[['index','year']].last()

last_values.groupby('year').max()

,index
year,
2017,14873


In [75]:
first_values.unique()

array([2018, 2017, 2016])